In [6]:
import argparse
import base64
import io
import logging
from typing import Dict

import json
import time

# pip install kfserving==0.3.0 -> dockerfile
# pip install numpy -> dockerfile
import numpy as np

# pip install opencv-python -> dockerfile
from PIL import Image
from PIL import ImageFilter

# for MobileNet Prediction Parsing
from tensorflow.keras.applications.mobilenet import decode_predictions
import tensorflow as tf

In [23]:






# KFServing - Transformer
# Preprocess
t1 = time.time()
model = tf.keras.experimental.load_from_saved_model('./saved_models/')


t2 = time.time()
img = Image.open('./1mb.jpg')

img = img.rotate(90).rotate(90).rotate(90).rotate(90)
img = img.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.FLIP_TOP_BOTTOM)
img = img.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.FLIP_LEFT_RIGHT)
img = img.resize((224, 224))
img = np.array(img)
img = img/255
img = img.reshape(-1, 224, 224, 3)
t3 = time.time()

# KFServing - Predictor
logging.info('start predicting!')
yhat = model.predict(img)

t4 = time.time()

# KFServing - Transformer
label = decode_predictions(yhat)
label = label[0][0]
results = [label[1], str(round(label[2]*100, 2))+'%']
logging.info(results)
print(results)

t5 = time.time()

yhat2 = model.predict(img)

t6 = time.time()
print(t1, t2, t3, t4)
print(t2 - t1)
print(t3 - t2)
print(t4 - t3)
print(t5 - t4)
print(t6 - t5)
print(t4 - t2)


['lampshade', '48.34%']
1592962241.59185 1592962243.293895 1592962243.587174 1592962245.59712
1.70204496383667
0.29327893257141113
2.00994610786438
0.0005030632019042969
0.05605769157409668
2.303225040435791


['Siamese_cat', '38.95%']
4.014510154724121


['Siamese_cat', '38.95%']